In [1]:
import numpy as np
import tensorflow as tf
import utils9
import sys
import os
import pandas as pd
import pickle
import math
import time
import zhihu_rnn2_bn as rnn

In [2]:
word_dict = pickle.load(open('./ieee_zhihu_cup/word_dict' ,'r'))
char_dict = pickle.load(open('./ieee_zhihu_cup/char_dict' ,'r'))

In [3]:
cnn_seq_length = 60 
rnn_seq_length = 80
batch_size = 256
eval_batch_size = 2000
hidden_size = 180
lr = 0.005
reg_rate = 0.0001
epoch_num = 20
save_per_step = 1000
eval_per_step = 500
keep_prob = 0.5 
atn_hidden_size = 256
shuffle = True
ckpt_path = './models/'
num_sentences = 2
filter_sizes = [2,3,4,5,6,7]
num_filters = 256
decay_steps = 1000 
decay_rate = 0.90
vocab_size = len(word_dict) 
char_size = len(char_dict) 
embed_size = 128

summary_dir = './Log/'
max_score = 0.0
model_name = 'utils9-rnn-model'

In [4]:
utils = utils9.Utils(words_num=rnn_seq_length, chars_num=cnn_seq_length, batch_size=batch_size, 
                     eval_batch_size=eval_batch_size, dataset_dir='./ieee_zhihu_cup/', 
                     epoch_num=epoch_num, word_dict=word_dict, char_dict=char_dict, shuffle=True)

In [5]:
def train(sess, utils):
    # 生成数据
    batches = utils.generate_t_batch()
    eval_batches = utils.generate_e_batch()
    model = rnn.RNN_char_word(cnn_seq_length, rnn_seq_length, batch_size, eval_batch_size, hidden_size, 
                                lr, reg_rate, epoch_num, save_per_step, eval_per_step, keep_prob, 
                                atn_hidden_size, shuffle, ckpt_path, num_sentences, 
                                filter_sizes, num_filters, decay_steps, decay_rate, 
                                vocab_size, char_size, embed_size, is_train=True)
    sess.run(tf.global_variables_initializer())
    
    summary_writer = tf.summary.FileWriter(summary_dir, graph=tf.get_default_graph())
    
    for batch in batches:
        x_char, x_char_length, x_word, x_word_length, y = zip(*batch)
        batch_length = len(x_char)

        feed_dict = {model.cnn_input: x_char, 
                     model.rnn_input: x_word,
                     model.output_keep_prob: keep_prob,
                     model.batch_length: [batch_length],
                     model.labels: y,
                     model.word_input_len: x_word_length,
                     model.char_input_len: x_char_length}
        
        train_loss, step, summary, _= sess.run([model.loss, model.global_step, model.merged, model.train_op], 
                                                feed_dict) 
        summary_writer.add_summary(summary, step)
        if (step % eval_per_step == 0 and step > 10000):
            path = model.saver.save(sess, "models/"+model_name, global_step=step)
            print("Saved model checkpoint to {}".format(path))  
            print ("Step:",step)
            print ("Train loss:",train_loss)
            
            score_list = []
            eval_loss_list = []
            for i in range(40):
                x_char, x_char_length, x_word, x_word_length, y, real_labels = zip(*(eval_batches.next()))
                score, eval_loss = do_eval(utils, sess, model, x_char, x_char_length, 
                                           x_word, x_word_length, y, real_labels)
                score_list.append(score)
                eval_loss_list.append(eval_loss)
            print("avg eval loss:", np.mean(eval_loss_list))
            save_best_model(score_list, step) 
            
# 在验证集上做验证，报告损失、精确度
def do_eval(utils, sess, model, x_char, x_char_length, x_word, x_word_length, y, real_labels):
    predict_top_5 = tf.nn.top_k(model.logits, k=5)
    batch_length = len(x_char)
    feed_dict = {model.cnn_input: x_char, 
                 model.rnn_input: x_word,
                 model.output_keep_prob: 1.0,
                 model.batch_length: [batch_length],
                 model.labels: y,
                 model.word_input_len: x_word_length,
                 model.char_input_len: x_char_length}
    curr_eval_loss, predict_5 = sess.run([model.loss, predict_top_5], feed_dict)
#     print ("Evaluation loss:",curr_eval_loss)
#     print ("real_labels:",real_labels[:5])
#     print ("predict:",predict_5[1][:5])
#     print ("predict:",predict_5[0][:5])
    predict_label_and_marked_label_list = []
    for predict,label in zip(predict_5[1],real_labels):
        predict_label_and_marked_label_list.append((list(predict),list(label)))
    score = utils.eval(predict_label_and_marked_label_list)
    return score, curr_eval_loss
#     print("score:",score)
#     print "--------------------Parting Line---------------------"
    
def predict(utils, restore=True):
    with tf.Session() as sess:
        if restore:
            model = rnn.RNN_char_word(cnn_seq_length, rnn_seq_length, batch_size, eval_batch_size, hidden_size, 
                                        lr, reg_rate, epoch_num, save_per_step, eval_per_step, keep_prob, 
                                        atn_hidden_size, shuffle, ckpt_path, num_sentences, 
                                        filter_sizes, num_filters, decay_steps, decay_rate, 
                                        vocab_size, char_size, embed_size, is_train=False)
            model.saver.restore(sess=sess, save_path=tf.train.latest_checkpoint(ckpt_path))
        predict_top_5 = tf.nn.top_k(model.logits, k=5)
        pred_batches = utils.generate_p_batch()
        for count, pred_batch in enumerate(pred_batches):
            sys.stdout.write("Count %d\r" % count)
            sys.stdout.flush()
            x_char, x_char_length, x_word, x_word_length = zip(*(pred_batch))
            batch_length = len(x_char)
            feed_dict = {model.cnn_input: x_char, 
                         model.rnn_input: x_word,
                         model.output_keep_prob: 1.0,
                         model.batch_length: [batch_length],
                         model.word_input_len: x_word_length,
                         model.char_input_len: x_char_length}
            predict_5 = sess.run(predict_top_5, feed_dict=feed_dict)
            if count == 0:
                predict = predict_5[1]
            else:
                predict = np.concatenate((predict,predict_5[1]))
        np.savetxt("./Result/predict.txt",predict,fmt='%d')
        
def save_best_model(score_list, step):
    global max_score
    avg_score = np.mean(score_list)
    avg_score_str = "%d steps avg score: %f\n" % (step, avg_score) 
    file_name = 'avg_loss_file'
    save_path = './best_model/'
    if os.path.exists(save_path):
        wr = open(save_path + file_name, 'a')
        wr.write(avg_score_str)
        if avg_score > max_score:
            max_score = avg_score
            os.system('rm ./best_model/%s*' % model_name)
            os.system('cp ./models/%s-' % model_name + str(step) + '* ' + save_path)
    else:
        os.mkdir(save_path)
        os.system('cp ./models/%s-' % model_name + str(step) + '* ' + save_path)   
    print(avg_score_str)
    print "--------------------Parting Line---------------------"

In [6]:
with tf.Session() as sess:
    train(sess, utils)

Saved model checkpoint to models/utils9-rnn-model-10500
('Step:', 10500)
('Train loss:', 0.0044589015)
('avg eval loss:', 0.0041237078)
10500 steps avg score: 0.382478

--------------------Parting Line---------------------
Saved model checkpoint to models/utils9-rnn-model-11000
('Step:', 11000)
('Train loss:', 0.004385896)
('avg eval loss:', 0.0041215597)
11000 steps avg score: 0.383682

--------------------Parting Line---------------------
Saved model checkpoint to models/utils9-rnn-model-11500
('Step:', 11500)
('Train loss:', 0.0040990356)
('avg eval loss:', 0.0041202181)
11500 steps avg score: 0.382581

--------------------Parting Line---------------------
Saved model checkpoint to models/utils9-rnn-model-12000
('Step:', 12000)
('Train loss:', 0.0040471544)
('avg eval loss:', 0.0041349111)
12000 steps avg score: 0.383895

--------------------Parting Line---------------------
Saved model checkpoint to models/utils9-rnn-model-12500
('Step:', 12500)
('Train loss:', 0.004206622)
('avg e

KeyboardInterrupt: 

In [6]:
print("params conf. file store success!")
predict(utils)

params conf. file store success!
INFO:tensorflow:Restoring parameters from ./models/utils9-rnn-model-50000
('predict_set_num_batches:', 109)
